## Modelo para Classificação de Textos

 - Implementar uma tarefa de classificação de texto usando o machine learning. 
 - Aprender a criar recursos básicos baseados em NLP que podem ser criados a partir do texto e testará o modelo no conjunto de dados de teste para avaliar seu desempenho.
 
A tarefa é criar um modelo de aprendizado de máquina para ** classificar ** se um determinado tweet é ** discurso de ódio ** ou ** não **. Mais explicações serão dadas a seguir.
 
### Sumário

1. Sobre o conjunto de dados
2. Limpeza de Texto
3. Feature Engineering
4. Treine um modelo de ML para classificação de texto
5. Avalie o modelo ML
6. Conclusão

### 1. Sobre o conjunto de dados

O conjunto de dados usado é o de ** Detecção de ódio ** nos tweets das pessoas. Você pode baixá-lo em [aqui.] (http://trainings.analyticsvidhya.com/asset-v1:AnalyticsVidhya+NLP101+2018_T1+type@asset+block@final_dataset_basicmlmodel.csv)
Carregar o conjunto de dados usando pandas e dar uma olhada rápida em alguns exemplos de tweets.


In [1]:
#Carregar o dataset
import pandas as pd

dataset = pd.read_csv('final_dataset_basicmlmodel.csv')
dataset.head()

FileNotFoundError: [Errno 2] File b'final_dataset_basicmlmodel.csv' does not exist: b'final_dataset_basicmlmodel.csv'

** Coisas a observar **
  - ** label ** é a coluna que contém a variável objetivo (target value) que deve ser previsto. '1' significa que é um discurso de ódio e '0' significa que não é.
  - ** Tweet ** é a coluna que contém o texto do Tweet. Estes são os principais dados nos quais as técnicas de NLP serão aplicadas.
 
Examinando alguns dos tweets.

In [ ]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
    print(index+1,".",tweet)

** Nota: - Ruído presente nos Tweets

- Observe que há muitas hashtags presentes nos tweets do símbolo do formulário #, seguidas pelo texto. Particularmente, não precisa do símbolo #, então será retirado
- Além disso, existem símbolos estranhos como â e ð no tweet 4. Na verdade, são caracteres unicode que estão presentes no conjunto de dados os quais serão retirados porque eles não acrescentam nada significativo.
- Existem também numerais e porcentagens.

### 2. Limpeza de Dados

Limpar o ruído no conjunto de dados.

In [ ]:
import re

#Limpar ruídos no texto
def clean_text(text):
    #Filtrar somente alfabetos
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove caracter Unicode
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    #Converte em lowercase para manter a consistência
    text = text.lower()
       
    return text

In [ ]:
dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))
dataset.head()

### 3. Feature Egineering

- Feature Engineering é a ciência (e a arte) de extrair mais informações dos dados existentes. Não será adicionando novos dados aqui, mas sim melhorar os dados existentes para ficarem mais úteis.
- O modelo de machine learning não entende o texto diretamente; portanto, será criado recursos numéricos que repercutem no texto subjacente.
- Neste módulo serão vistos recursos muito básicos baseados em NLP e, à medida que se avança outras maneiras mais complexas e eficientes de fazer o mesmo serão mostrados.

In [ ]:
#Lista exaustiva de palavras irrelevantes no idioma inglês. Queremos focar menos nisso, 
#então em algum momento teremos que filtrar
STOP_WORDS = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and',
              'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
              'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did',
              "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever',
              'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how',
              "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it',
              "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself',
              'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's",
              'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs',
              'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't",
              'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
              "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't",
              'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

#Gera frequência de palavras (word frequency)
def gen_freq(text):
    #Armazena numa list de palavras
    word_list = []

    #Loop sobre todos os tweets e extrai as palavras da word_list
    for tw_words in text.split():
        word_list.extend(tw_words)

    #Cria frequências de palavras usando word_list
    word_freq = pd.Series(word_list).value_counts()
    
    #Tira stopwords durante o cálculo de frequências
    word_freq = word_freq.drop(STOP_WORDS, errors='ignore')
    
    return word_freq

#Verifica se um termo de negação está presente no texto
def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
    else:
        return 0

#Verifique se uma das 100 palavras raras está presente no texto
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
    else:
        return 0

#Verifique se as palavras de prompt estão presentes
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
    else:
        return 0

In [ ]:
word_freq = gen_freq(dataset.clean_text.str)
#100 mais raras palavras no  dataset
rare_100 = word_freq[-100:]
#Número de palavras em um tweet
dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))
#Negação presente ou não
dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x: any_neg(x))
#Prompt presente ou não
dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x: is_question(x))
#Qualquer uma das 100 palavras raras presentes ou não
dataset['any_rare'] = dataset.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))
#Caractere contado no tweet
dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))

In [ ]:
#Top 10  palavras comuns
gen_freq(dataset.clean_text.str)[:10]

In [ ]:
dataset.head()

### Dividindo o conjunto de dados em Train-Test

  - O conjunto de dados é dividido em conjuntos de treino e teste, para que seja avaliado o desempenho do modelo em dados não vistos.
  - O modelo será treinado apenas no conjunto `train` e fará previsões no conjunto` test` cujos dados o modelo nunca viu. Isso garantirá que se tenha uma maneira adequada de testar o modelo.

In [ ]:
from sklearn.model_selection import train_test_split

X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=27)

### 4. Treinar um modelo de ML para classificação de texto

Agora que o conjunto de dados está pronto, é hora de treinar um modelo de Machine Learning. Será usado um classificador ** Naive Bayes ** do `sklearn`.

In [ ]:
from sklearn.naive_bayes import GaussianNB

#Initialize GaussianNB classifier
model = GaussianNB()
#Fit the model on the train dataset
model = model.fit(X_train, y_train)
#Make predictions on the test dataset
pred = model.predict(X_test)

In [ ]:
from sklearn.naive_bayes import BernoulliNB

#Initialize GaussianNB classifier
model2 = BernoulliNB()
#Fit the model on the train dataset
model2 = model2.fit(X_train, y_train)
#Make predictions on the test dataset
pred2 = model2.predict(X_test)

### 5. Avaliar o modelo ML

Treinar o modelo em dados não vistos anteriormente: ** X_test ** e ** y_test ** conjuntos criados anteriormente para verificar a precisão do modelo.

In [ ]:
from sklearn.metrics import accuracy_score

print("    Gauss Accuracy:", accuracy_score(y_test, pred)*100, "%")
print("Bernoulli Accuracy:", accuracy_score(y_test, pred2)*100, "%")

### 6. Conclusão

** Nota-se ** como foram usados os recursos básicos de NLP, a precisão da classificação e as pontuações f1 não são tão impressionantes. O objetivo deste exercício foi  de familiarizar com o processo de construção do modelo.